# Instalasi

Sebelum memulai penggunakan Google Gemini, kita harus menginstalasi python package terlebih dahulu.

Menggunakan `Python 3.9+` , instal google-genai paket menggunakan perintah pip berikut:

In [ ]:
!pip install -q -U "google-genai>=1.0.0"

Gunakan metode `generateContent` untuk mengirim permintaan ke API Gemini.

In [20]:
from google.colab import userdata
API_KEY = userdata.get('GEMINI')

In [ ]:
from google import genai

# masukan API key kalian
client = genai.Client(api_key=API_KEY)

MODEL_ID= 'gemini-2.5-flash'
response= client.models.generate_content(
  model=MODEL_ID,
  contents=[
    'Explain how AI works.',
  ]
)

print(response.text)

Agar tampilannya lebih rapi, kalian dapat menggunakan fungsi `Markdown`

In [ ]:
from IPython.display import Markdown

Markdown(response.text)

Untuk mengecek list model yang ada pada Gemini, kita bisa menggunakan perintah berikut

In [ ]:
#cek list model dari gemini
for m in client.models.list():
  print(m.name)

Secara default, model mengembalikan respons setelah menyelesaikan seluruh proses pembuatan teks. Anda dapat memperoleh interaksi yang lebih cepat dengan tidak menunggu seluruh hasil, dan sebagai gantinya menggunakan streaming untuk menangani hasil parsial.

```python
from google import genai

client = genai.Client()
response = client.models.generate_content_stream(
    model="gemini-2.0-flash", contents="Write a story about a magic backpack."
)
for chunk in response:
    print(chunk.text)
    print("_" * 80)
```

In [ ]:
response = client.models.generate_content_stream(
    model="gemini-2.5-flash", contents="Write a story about a magic backpack."
)
for chunk in response:
    print(chunk.text)
    print("_" * 80)

# Generate text from text-and-image input

Kita akan menggunkan PIL untuk membaca gambar kemudian menambahkan prompt untuk menghasilkan text.

In [ ]:
!curl -o salon_price_list.jpg https://marketplace.canva.com/EAFhLHB9S8g/1/0/900w/canva-pink-putih-minimalist-salon-price-list-28N6AZMRZKI.jpg

In [ ]:
img

In [ ]:
import PIL


img = PIL.Image.open("salon_price_list.jpg")
prompt= '' # tanpa prompt
model_id='gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    contents=[img, prompt]
)

Markdown(response.text)

In [ ]:
prompt= 'buatlah table dari response tersebut' # dengan prompt
response = client.models.generate_content(
  model=model_id,
  contents=[img, prompt]
)

Markdown(response.text)

In [ ]:
prompt= '''buatlah tabel dari response tersebut,
buatlah code python untuk membuat dataframe dari tabel tersebut'''
response = client.models.generate_content(
  model=model_id,
  contents=[img, prompt]
)

Markdown(response.text)

Untuk mempelajari lebih detail berkaitan dengan pengolahan Image dengan Gemini, Anda dapat melihat pada referensi berikut:
- [Explore vision capabilities with the Gemini API](https://ai.google.dev/gemini-api/docs/vision?lang=python)

# Document Understanding

Google Gemini juga dapat kita gunakan untuk mengelolah file atau dokumen dalam berbagai macam format, misalkan format pdf.

Untuk mempelajari lebih detail berkaitan dengan pengolahan Dokumen dengan Gemini, Anda dapat melihat pada referensi berikut:
- [Explore document processing capabilities with the Gemini API](https://ai.google.dev/gemini-api/docs/document-processing?hl=id&lang=python)

In [ ]:
from google.genai import types
import httpx
import base64

doc_url = "https://discovery.ucl.ac.uk/id/eprint/10089234/1/343019_3_art_0_py4t4l_convrt.pdf"  # Replace with the actual URL of your PDF

# Retrieve and encode the PDF
doc_data = httpx.get(doc_url).content

prompt = "Summarize this document"
response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=doc_data,
        mime_type='application/pdf',
      ),
      prompt])
print(response.text)


# Audio Understanding

Google Gemini juga dapat kita gunakan untuk mengelolah Audio dalam berbagai macam format, misalkan format MP3,WAV dan format lainnya. Berikut adalah contoh codenya:

```python
your_audio_file = client.files.upload(file='sample.mp3')
response = client.models.generate_content(
  model=MODEL_ID,
  contents=[
    'Describe this audio clip.',
    your_audio_file,
  ]
)

print(response.text)
```

Untuk mempelajari lebih detail berkaitan dengan pengolahan Audio dengan Gemini, Anda dapat melihat pada referensi berikut:
- [Explore audio capabilities with the Gemini API](https://ai.google.dev/gemini-api/docs/audio?lang=python)

In [32]:
URL = "https://storage.googleapis.com/generativeai-downloads/data/State_of_the_Union_Address_30_January_1961.mp3"
!wget -q $URL -O sample.mp3

In [ ]:
your_audio_file= client.files.upload(file='sample.mp3')
your_audio_file

In [ ]:
response = client.models.generate_content(
  model=model_id,
  contents=[
    'Listen carefully to the following audio file. Provide a brief summary.',
    your_audio_file,
  ]
)

print(response.text)

# Chat Conversation

Gemini SDK memungkinkan Anda mengumpulkan beberapa putaran pertanyaan dan respons, yang memungkinkan pengguna untuk melangkah secara bertahap menuju jawaban atau mendapatkan bantuan untuk masalah yang terdiri dari beberapa bagian. Fitur SDK ini menyediakan antarmuka untuk melacak **riwayat percakapan**, tetapi di balik layar menggunakan metode `generateContent`.

In [ ]:
chat = client.chats.create(model="gemini-2.5-flash")

response = chat.send_message("Dalam satu kalimat, jelaskan cara kerja komputer kepada anak kecil.")
Markdown(response.text)

Komputer itu seperti otak pintar yang sangat cepat; ia menerima apa yang kamu suruh (misalnya, klik atau ketik), memikirkannya, lalu menunjukkan hasilnya padamu di layar.

In [ ]:
response = chat.send_message("Jelaskan cara membuat otomatisasi email")
Markdown(response.text)

Kita dapat mengecek history atau riwayat chat dengan perintah berikut.

In [ ]:
for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

# Generate Images

Pastikan Anda menggunakan model dan versi yang didukung untuk pembuatan gambar:

- Untuk **Gemini**, gunakan Pembuatan Gambar Pratinjau Flash Gemini 2.0.
- Untuk **Imagen**, gunakan Imagen 3. Perhatikan bahwa model ini hanya tersedia di Paket berbayar.



Lebih detailnya Anda dapat lihat pada referensi berikut:
[Generate Images](https://ai.google.dev/gemini-api/docs/image-generation?hl=id)

In [ ]:
from IPython.display import Image, Markdown

response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents='Hi, can create a 3d rendered image of a cat with wings and a top hat flying over a happy futuristic scifi city with lots of greenery?',
    config=types.GenerateContentConfig(
        response_modalities=['Text', 'Image']
    )
)

for part in response.candidates[0].content.parts:
  if part.text is not None:
    display(Markdown(part.text))
  elif part.inline_data is not None:
    mime = part.inline_data.mime_type
    print(mime)
    data = part.inline_data.data
    display(Image(data=data))